# Surface Based Group Analysis

This document includes text and images from the FreeSurfer tutorials. The full official FreeSurfer tutorials are available here:
https://surfer.nmr.mgh.harvard.edu/fswiki/Tutorials


## Introduction

This tutorial is designed to introduce you to the "command-line" group analysis stream in FreeSurfer. While this tutorial shows you how to perform a surface-based thickness study, it is important to realize that most of the concepts learned here apply to any group analysis in FreeSurfer, surface, volume, thickness or fMRI. Here are some useful Group Analysis links you might want to refer back to at a later time: 

https://surfer.nmr.mgh.harvard.edu/fswiki/FsgdFormat

https://surfer.nmr.mgh.harvard.edu/fswiki/FsgdExamples

https://surfer.nmr.mgh.harvard.edu/fswiki/DodsDoss


Before we go any further, we will import some general libraries and set up important paths:



In [ ]:
#import necessary libraries

import os
import subprocess
from pathlib import Path
from IPython.display import display, HTML
import shutil

# Set up paths
TUTORIAL_DATA = "/home/cognestic/COGNESTIC/04_Structural_MRI/CorticalThickness/tutorial_data_20190918_1558"
SUBJECTS_DIR = f"{TUTORIAL_DATA}/buckner_data/tutorial_subjs/group_analysis_tutorial"

# Set environment variables for subprocess commands
os.environ["TUTORIAL_DATA"] = str(TUTORIAL_DATA)
os.environ["SUBJECTS_DIR"] = str(SUBJECTS_DIR)

## Set up a terminal window

Open a terminal windown and set up the SUBJECTS_DIR environment variable by using the following command:

**export SUBJECTS_DIR=/home/cognestic/COGNESTIC/04_Structural_MRI/CorticalThickness/tutorial_data_20190918_1558/buckner_data/tutorial_subjs/group_analysis_tutorial**

And now change directory to the COGNESTIC group analysis tutotial data folder:

**cd $SUBJECTS_DIR**

## The Dataset

The data used for this tutorial is 40 subjects from Randy Buckner's lab. It consists of males and females, ages 18 to 93. You can see the demographics here:

https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/GroupAnalysisDemographics 

You will perform an analysis looking for the effect of age on cortical thickness, accounting for the effects of gender in the analysis. 

## General Linear Model (GLM) DODS Setup

### Design Matrix/FSGD File

In this tutorial, we will model the thickness as a straight line. A line has two parameters: an intercept (or offset) and a slope. For this example: 

1.	The slope is the change of thickness with age. 
2.	The intercept/offset is interpreted as the thickness at age=0. 

Parameter estimates are also called "regression coefficients" or "betas". To account for effects of gender, we will model each sex with its own line, meaning that there will be four linear parameters: 

1.	Intercept for Females 
2.	Intercept for Males 
3.	Slope for Females 
4.	Slope for Males 

In FreeSurfer, this type of design is called DODS (for "Different-Offset, Different-Slope"). You can either create your own design matrices, or, if you specify your design as a FreeSurfer Group Descriptor File (FSGD), FreeSurfer will create the design matrices for you. 

The FSGD file is a simple text file you create, it is not generated by FreeSurfer. See this page for the format:

https://surfer.nmr.mgh.harvard.edu/fswiki/FsgdFormat

The demographics page also has an example FSGD file for this data:

https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/GroupAnalysisDemographics

For this tutorial, the FGDS we need already exists (gender_age.fsgd) so that you can continue with the exercises. 

You can open the FSGD file for this tutorial (gender_age.fsgd) in a text editor such as gedit (for Linux) or open -e (for Macs), or you can browse it below:


In [ ]:
#Display FSDG file gender_age.fsgd 

fsgd_path = Path(SUBJECTS_DIR) / 'glm' / 'gender_age.fsgd'

# Display contents
if fsgd_path.exists():
    print(f"Contents of {fsgd_path}:\n")
    with open(fsgd_path, 'r') as f:
        content = f.read()
    print(content)
else:
    print(f"File not found: {fsgd_path}")

### Contrasts
A contrast is a vector that embodies the hypothesis we want to test. In this case, we wish to test the change in thickness with age, after removing the effects of gender. To do this, create a simple text file with the following numbers (if you're using the tutorial data, this has already been done for you): 

0 0 0.5 0.5

**Notes**: 

1.	Remember that for an analysis between two groups each with an intercept of b and a slope of m, the contrast matrix will be in the format [b1 b2 m1 m2]. 
2.	There is one value for each parameter (so 4 values total). 
3.	The intercept/offset values (b1, b2) are 0 (nuisance). 
4.	The slope values (m1, m2) are 0.5 so as to average the Female and Male slopes. 
5.	You'll find we created the contrast matrix for you already for this tutorial. It's a file called lh-Avg-thickness-age-Cor.mtx. You will need to create your own contrast matrix when testing your own hypotheses on your data. FreeSurfer doesn't automatically create this file. There are examples of FSGD files and contrast matrices for many different hypotheses here. 

### Exercise 1: Display the contrast file

1. Open a new code cell below, and write code to display the lh-Avg-thickness-age-Cor.mtx file, which is located in the folder $SUBJECTS_DIR/glm.

## Assemble the Data (mris_preproc)
Assembling the data simply means:

1.	Resampling each subject's data into a common space. 
2.	Concatenating all the subjects' into a single file. 
3.	Spatial smoothing (can be done between 1 and 2). 


### Uncached Data
In the case that you have not cached the data, you can use the two commands below. The commands below only create output for unsmoothed data and data smoothed to 10mm FWHM. 



In [ ]:
#Set up paths
GLM_DIR = Path(SUBJECTS_DIR) / 'glm'
GLM_COGNESTIC_DIR = Path(SUBJECTS_DIR) / 'glm_cognestic'

# Create glm_cognestic directory if it doesn't exist
GLM_COGNESTIC_DIR.mkdir(parents=True, exist_ok=True)

# Copy .fsgd and .mtx files
for ext in ("*.fsgd", "*.mtx"):
    for file in GLM_DIR.glob(ext):
        shutil.copy(file, GLM_COGNESTIC_DIR / file.name)

# Change to glm_cognestic directory
os.chdir(GLM_COGNESTIC_DIR)

# Run mris_preproc with output suppressed
#This will take a few minutes to run

with open(os.devnull, 'w') as DEVNULL:
    subprocess.run([
        "mris_preproc",
        "--fsgd", "gender_age.fsgd",
        "--target", "fsaverage",
        "--hemi", "lh",
        "--meas", "thickness",
        "--out", "lh.gender_age.thickness.00.mgh"
    ], stdout=DEVNULL, stderr=DEVNULL, check=True)

# Confirmation message
print("mris_preproc complete — output saved to lh.gender_age.thickness.00.mgh")

**Notes:** 

1.	The command above resamples each subject's left hemisphere data to fsaverage. 
2.	The output is lh.gender_age.thickness.00.mgh, which is unsmoothed. 


Next, we will apply smoothing, using a 10mm FWHM kernel in the example below:

In [ ]:
# Run mri_surf2surf with smoothing
subprocess.run([
    "mri_surf2surf",
    "--hemi", "lh",
    "--s", "fsaverage",
    "--sval", "lh.gender_age.thickness.00.mgh",
    "--fwhm", "10",
    "--cortex",
    "--tval", "lh.gender_age.thickness.10.mgh"
], check=True)

# Confirmation message
print("mris_surf2surf complete — output saved to lh.gender_age.thickness.10.mgh")

**Notes:**

1.	This smooths each subject's resampled data by 10mm FWHM. 
2.	"--cortex" means only smooth areas in cortex (exclude medial wall). This is automatically done with qcache. You can also specify other labels. 
3.	"--sval" is the name of the source subject found in the $SUBJECTS_DIR. This input data must be sampled onto this subject's surface. 
4.	"--tval" is the name of the file where the data on the target surface will be stored. 
5.	Output is lh.gender_age.thickness.10.mgh. 

## GLM Analysis (mri_glmfit)

With the pre-processing done, the next step is to fit a GLM to the data:


In [ ]:
#run mri_glmfit to perform the group analysis
# This command fits a linear model to the data, using the specified contrast matrix and FSG

subprocess.run([
    "mri_glmfit",
    "--y", "lh.gender_age.thickness.10.mgh",
    "--fsgd", "gender_age.fsgd", "dods",
    "--C", "lh-Avg-thickness-age-Cor.mtx",
    "--surf", "fsaverage", "lh",
    "--cortex",
    "--glmdir", "lh.gender_age.glmdir"
], check=True)

# Confirmation message
print("mri_glmfit complete — output saved to lh.gender_age.glmdir")

**Notes:** 

1.	Input is lh.gender_age.thickness.10.mgh. 
2.	Same FSGD used as with mris_preproc. Maintains subject order! 
3.	DODS is specified (it is the default). 
4.	Only one contrast is used (lh-Avg-thickness-age-Cor.mtx), but you can specify multiple contrasts. 
5.	"--cortex" specifies that the analysis only be done in cortex (ie, medial wall is zeroed out). Other labels can be used. 
6.	The output directory is lh.gender_age.glmdir. 
7.	Should only take about 1min to run. 
8.	For more information about mri_glmfit, click here 

### Model outputs

When this command is finished the glm directory will contain a sub-directory called **lh.gender_age.glmdir**. There will be a number of output files in this directory, as well as other subdirectories (you may also see some temporary directories, with names starting with tmp, or log files, which you can ignore):

**beta.mgh** -- all parameter estimates (surface overlay)

**dof.dat**  -- degrees of freedom (text)

**fwhm.dat** -- average FWHM of residual (text)

**lh-Avg-thickness-age-Cor** -- contrast subdirectory

**mask.mgh** -- binary mask (surface overlay)

**mri_glmfit.log** -- log file (text, send this with bug reports)

**rstd.mgh** -- residual standard deviation (surface overlay)

**rvar.mgh** -- residual variance (surface overlay)

**sar1.mgh** -- residual spatial AR1  (surface overlay)

**surface** -- the subject and hemisphere used for this analysis (text)

**Xg.dat** -- design matrix  (text)

**X.mat** -- design matrix (MATLAB format)

**y.fsgd** -- copy of input FSGD file  (text)


**Notes:** 

1.	The DOF is the degrees of freedom for the analysis which has important implications for calculating the p-value. 
2.	The FWHM is a measure of the smoothness of the data. Part of this comes from the applied smoothing (eg, 5mm FWHM) and part of the smoothness comes from the inherent smoothness in the data. When correcting for multiple comparisons, the final FWHM must be taken into account (but this is done automatically). 

### Contrast outputs

There will be a subdirectory for each contrast that you specify. The name of the directory will be that of the contrast matrix file (without the .mtx extension). For example, inside the **lh-Avg-thickness-age-Cor** directory you will find the following files:

**C.dat** -- original contrast matrix (text)

**cnr.mgh** -- contrast-to-noise ratio (surface overlay)

**efficiency.dat** -- statistical efficiency for the contrast (text)

**F.mgh** -- F ratio of contrast  (surface overlay)

**gamma.mgh** -- contrast effect size (surface overlay)

**gammavar.mgh** --contrast variance (surface overlay)

**maxvox.dat** -- voxel with the maximum statistic (text)

**pcc.mgh** -- partial (pearson) correlation coefficient (surface overlay)

**sig.mgh** -- significance, -log10(pvalue), uncorrected (surface overlay)

**z.mgh** -- z-stat that corresponds to the significance (surface overlay)


### Visualising the results

In order to view the uncorrected significance map, we need to use freeview. Return to the terminal window you set up at the start of this tutorial and run commands described below.

First, make sure you are in the correct directory: 

**cd $SUBJECTS_DIR/glm_cognestic**

Then, run this command to visualize the data: 

**freeview -f $SUBJECTS_DIR/fsaverage/surf/lh.inflated:annot=aparc.annot:annot_outline=1:overlay=lh.gender_age.glmdir/lh-Avg-thickness-age-Cor/sig.mgh:overlay_threshold=4,5 -viewport 3d -layout 1**

This command opens the left hemisphere inflated surface with the aparc annotation shown as an outline. The overlay sig.mgh is also loaded with a threshold of 4. In FreeSurfer the significance values are defined as:

$ \text{sig} = -\log_{10}(\text{p-value}) $

which means that a threshold of 4 corresponds to a p-value threshold of $10^{-4}$.

You should see something similar to this: 

![Example of significance map overlay](uncorrected_sig_map.png)

This figure displays the correlation between age and cortical thickness with a threshold of 4. 

**Notes:** 

1.	The threshold is set to 4, meaning vertices with p<0.0001, uncorrected, will have color. In this case, any vertex with a value under 4 will not be displayed in color, however the value will still be readable in the cursor/mouse section of freeview. 
2.	The lower threshold sets the minimum sigificance that a voxel must meet in order to be visible. It should be set such that only likely true effects will be seen. You can kind of think of it as a filter that removes voxels where no effect is present. The upper is just used for visualization purposes. If you set it very close to the min, then all voxels will appear to be the same color. If you set it higher, then you can get some gradation due to the strength of the effect. So the min removes voxels without an effect and the max allows you to see how the strength of the effect varies across the brain. 
3.	Blue represents a negative correlation (i.e. thickness decreases with age), red represents positive correlation (i.e. thickness increases with age). 
4.	In this example, the sig.mgh is used as the overlay and in practice this is what is normally used. However, the pcc.mgh can also be used as the overlay if it was created in the analysis.


### Exercise 2: Explore the GLM output

1. Click on a point in the Precentral Gyrus. What is its value? What does it mean?
2. Viewing the medial surface, change the overlay threshold to something very, very low (say, .01), by clicking **Configure** (under the **Overlay** dropdown menu that says sig.mgh) and set the Min value to 0.01. Click **Apply**, or check the Automatically apply changes checkbox.
3. Close freeview when you are finished.

You should see something like this: 

![Example of significance map overlay with different threshold](uncorrected_sig_map_2.png)


This figure displays the correlation between age and cortical thickness with a threshold of 0.01. 

**Notes:** 

1.	Almost all of the cortex now has color. 
2.	The non-cortical areas are still blank (0) because they were excluded with --cortex in mri_glmfit above. 

All the surface overlays created by mri_glmfit, not just the significance map, can also be inspected in freeview. Simply replace the path to the sig.mgh file in the command above to the surface overlay that you would like to see. For example, this will display the F ratio instead of the significance map: 

**freeview -f $SUBJECTS_DIR/fsaverage/surf/lh.inflated:annot=aparc.annot:annot_outline=1:overlay=lh.gender_age.glmdir/lh-Avg-thickness-age-Cor/F.mgh:overlay_threshold=20,50 -viewport 3d -layout 1**


## Clusterwise Correction for Multiple Comparisons (Permutation)

The method used is based on: False positive rates in surface-based anatomical analysis. Greve and Fischl, NeuroImage (2017):

https://www.ncbi.nlm.nih.gov/pubmed/29288131


### Introduction

To perform a cluster-wise correction for multiple comparisons, we will run a permutation simulation. The simulation is a way to get a measure of the distribution of the maximum cluster size under the null hypothesis. First, you run the analysis to get uncorrected maps. Then the permutation simulation is done by iterating over the following steps: 

1.	Permute the design matrix 
2.	Analyze the permuted data, including computing contrasts and sig maps 
3.	Threshold sig map (cluster forming threshold (CFT) and sign). 
4.	Find clusters in thresholded map. 
5.	Record area of maximum cluster. 
6.	Repeat over desired number of iterations (usually 1,000). 

In FreeSurfer, this information is stored in a simple text file called a CSD (Cluster Simulation Data) file that you can find in the glmdir output folder (subfolder csd) after running mri_glmfit-sim. 

Once we have the distribution of the maximum cluster size, we correct for multiple comparisons by: 

1.	Going back to the original, uncorrected data. 
2.	Thresholding using same level and sign. 
3.	Finding clusters in thresholded map. 
4.	For each cluster, p = probability of seeing a maximum cluster that size or larger during simulation. 

Re-run the initial analysis to get uncorrected results:


In [ ]:
#run mri_glmfit with eres-save option to save residuals
# This command performs the same analysis but saves the residuals of the model fit

subprocess.run([
    "mri_glmfit",
    "--y", "lh.gender_age.thickness.10.mgh",
    "--fsgd", "gender_age.fsgd", "dods",
    "--C", "lh-Avg-thickness-age-Cor.mtx",
    "--surf", "fsaverage", "lh",
    "--cortex",
    "--glmdir", "lh.gender_age.glmdir",
    "--eres-save"
], check=True)

# Confirmation message
print("mri_glmfit complete — output saved to lh.gender_age.glmdir")

This is the same command that you ran before in the Group Analysis tutorial (note the --eres-save option needed for permutation simulation). 

### Run the simulation

All the permutation steps above, including the final correction, are performed with the command mri_glmfit-sim below. For a real analysis you would set the number of permutations to at least 1000, but this would take about 20 minute to run. Therefore the command below will run it with 100 permutations instead:


In [ ]:
# Run permutation simulation with 100 permutations only
subprocess.run([
    "mri_glmfit-sim",
    "--glmdir", "lh.gender_age.glmdir",
    "--perm", "100", "4.0", "abs",     #change number of permutation to 1000 for a "real life" analysis
    "--cwp", "0.05",
    "--2spaces",
    "--overwrite"

], check=True)

# Confirmation message
print("mri_glmfit-sim complete")

**Notes:** 

1.	Specify the same GLM directory (--glmdir). 
2.	Run a permuation simulation (--perm). 
3.	Vertex-wise/cluster-forming threshold of 4 (p < .0001). 
4.	Specify the sign ("neg" for negative, "pos" for positive, or "abs" for absolute/unsigned). 
5.	--cwp 0.05 : Keep clusters that have cluster-wise p-values < 0.05. To see all clusters, set to .999. 
6.	--2spaces : adjust p-values for two hemispheres (this assumes you will eventually look at the right hemisphere too). 
7.	--bg 1 : Do not run in parallel (N=1 means single thread). If you want to run in parallel to reduce the run time use --bg N where N is the number of threads 
8.	You can also use Permutation Analysis of Linear Models PALM 

### View the Corrected Results
In the contrast subdirectory, you will see the following new files: 

**perm.th40.abs.pdf.dat** -- probability distribution function of clusterwise correction

**perm.th40.abs.sig.cluster.mgh** -- cluster-wise corrected map (overlay)

**perm.th40.abs.sig.cluster.summary** -- summary of clusters (text)

**perm.th40.abs.sig.masked.mgh** -- uncorrected sig values masked by the clusters that survive correction

**perm.th40.abs.sig.ocn.annot** -- output cluster number (annotation of clusters)

**perm.th40.abs.sig.ocn.mgh** -- output cluster number (segmentation showing where each numbered cluster is)

**perm.th40.abs.sig.voxel.max.dat** -- maximum voxel-wise significance

**perm.th40.abs.sig.voxel.mgh** -- voxel-wise map corrected for multiple comparisons at a voxel (rather than cluster) level


First, look at the cluster summary:



In [ ]:
# Output cluster summary to terminal
summary_path = Path(GLM_COGNESTIC_DIR) / 'lh.gender_age.glmdir' / 'lh-Avg-thickness-age-Cor' / 'perm.th40.abs.sig.cluster.summary'

if summary_path.exists():
    print("\nCluster Summary:\n")
    with open(summary_path, "r") as file:
        print(file.read())
else:
    print("\nCluster summary file not found:", summary_path)

**Notes:** 

1.	This is a list of all the clusters that were found (25 of them). 
2.	The CWP column is the cluster-wise probability (the number you are interested in). It is a simple p (ie, NOT -log10(p)) that indicates the probability of a cluster. 
3.	For example, cluster number 1 has a CWP of p=.002. 
4.	For explanations of the other columns in the cluster summary, see below:

https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/MultipleComparisonsClusterSummaryV6.0Perm

Return to the terminal and run the commad below to visualise the cluster annotation in freeview: 

**freeview -f $SUBJECTS_DIR/fsaverage/surf/lh.inflated:overlay=lh.gender_age.glmdir/lh-Avg-thickness-age-Cor/perm.th40.abs.sig.cluster.mgh:overlay_threshold=2,5:annot=lh.gender_age.glmdir/lh-Avg-thickness-age-Cor/perm.th40.abs.sig.ocn.annot -viewport 3d -layout 1**

You should see clusters similar in shape to those pictured in the snapshots below. The color values associated with each cluster are arbitrary and may be different: 

![All clusters, regardless of significance](All_clusters.png)

**Notes:**

1.	These are all clusters, regardless of significance. 
2.	When you click on a cluster, the label will tell you the cluster number (eg, cluster-016) which is automatically generated. 

### Exercise 3: Explore the corrected results

1.	Find and click on cluster 1 (the largest cluster). What is it's signigicance value? What is the meaning of the negative significance?
2.	Turn off the annotation (under **Annotaion** select **Off** from the drop down menu). Did any clusters disappear? Why?
3.	You can change the cluster-wise threshold by first clicking on **Show outline only** underneath the **Annotation** drop down menu. Then click on **Configure** (underneath **Overlay**), and set the **Min** value to your desired level. Alternatively, you can drag the red flag to adjust the cluster-wise threshold. As you do this, clusters will appear or disappear from the surface. (If your cursor is in the Min text box, the red flag won't move. Click on another text box to be able to move the flag.)
4. Close freeview when you are finished. 


## Summary
By the end of this tutorial, you should know how to: 

•	Run the analysis to get the uncorrected maps using mri_glmfit 

•	Run the permutation simulations using mri_glmfit-sim 

•	Correct for multiple comparisons 

